In [4]:
import os
import random
import tensorflow as tf
import cv2
import numpy as np
import time
import csv
import glob
import shutil
import sys
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils.generic_utils import custom_object_scope
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.quantization.keras import quantize_config
from tensorflow_model_optimization.python.core.quantization.keras import quantizers

Thepath = "/Users/george/Desktop/ck+/Images"

LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

conv2d=3
conv2d_1=3
conv2d_2=3
activ=3
activ_1=3
activ_2=3
maxpool=3
maxpool_1=3
maxpool_2=3
flatten=2
dropout=2
dense=4
dense_1=2

#QuantizerClasses
class DefaultConv2DQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return [(layer.weights[i], MovingAverageQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))for i in range(0)]
     
    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
        [(layer.weights[i] == quantize_weights[0]) for i in range(0)]
        
    def set_quantize_activations(self, layer, quantize_activations):
        layer.activation = quantize_activations[0]

    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=8, per_axis=False, symmetric=False, narrow_range=False)]

    def get_config(self):
        return {}
       
class DefaultActivationQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return []
        
    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
        pass
        
    def set_quantize_activations(self, layer, quantize_activations):
        pass

    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=8, per_axis=False, symmetric=False, narrow_range=False)]

    def get_config(self):
        return {}
    
class DefaultMaxPool2DQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return []
        
    def get_activations_and_quantizers(self, layer):
        return []

    def set_quantize_weights(self, layer, quantize_weights):
        pass
        
    def set_quantize_activations(self, layer, quantize_activations):
        pass

    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=8, per_axis=False, symmetric=False, narrow_range=False)]

    def get_config(self):
        return {}    

class DefaultFlattenQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return []
        
    def get_activations_and_quantizers(self, layer):
        return []

    def set_quantize_weights(self, layer, quantize_weights):
        pass
        
    def set_quantize_activations(self, layer, quantize_activations):
        pass

    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=8, per_axis=False, symmetric=False, narrow_range=False)]

    def get_config(self):
        return {}
    
class DefaultDropoutQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return []
        
    def get_activations_and_quantizers(self, layer):
        return []

    def set_quantize_weights(self, layer, quantize_weights):
        pass
        
    def set_quantize_activations(self, layer, quantize_activations):
        pass

    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=8, per_axis=False, symmetric=False, narrow_range=False)]

    def get_config(self):
        return {} 
    
class DefaultDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return [(layer.kernel, MovingAverageQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]
     
    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
        layer.kernel = quantize_weights[0]
        
    def set_quantize_activations(self, layer, quantize_activations):
        layer.activation = quantize_activations[0]

    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=8, per_axis=False, symmetric=False, narrow_range=False)]

    def get_config(self):
        return {}
    
#ModifiedConv2DQuantizerClasses
class ModifiedConv2DQuantizeConfig(DefaultConv2DQuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return [(layer.weights[i], MovingAverageQuantizer(num_bits=conv2d, symmetric=True, narrow_range=False, per_axis=False))for i in range(0)]
    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=conv2d, symmetric=False, narrow_range=False, per_axis=False))]
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=conv2d, per_axis=False, symmetric=False, narrow_range=False)]
    
class ModifiedConv2D_1QuantizeConfig(DefaultConv2DQuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return [(layer.weights[i], MovingAverageQuantizer(num_bits=conv2d_1, symmetric=True, narrow_range=False, per_axis=False))for i in range(0)]
    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=conv2d_1, symmetric=False, narrow_range=False, per_axis=False))]
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=conv2d_1, per_axis=False, symmetric=False, narrow_range=False)]
    
class ModifiedConv2D_2QuantizeConfig(DefaultConv2DQuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return [(layer.weights[i], MovingAverageQuantizer(num_bits=conv2d_2, symmetric=True, narrow_range=False, per_axis=False))for i in range(0)]
    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=conv2d_2, symmetric=False, narrow_range=False, per_axis=False))]
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=conv2d_2, per_axis=False, symmetric=False, narrow_range=False)]
    
#ModifiedActivationQuantizerClasses    
class ModifiedActivationQuantizeConfig(DefaultActivationQuantizeConfig):
    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=activ, symmetric=False, narrow_range=False, per_axis=False))]
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=activ, per_axis=False, symmetric=False, narrow_range=False)]
    
class ModifiedActivation_1QuantizeConfig(DefaultActivationQuantizeConfig):
    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=activ_1, symmetric=False, narrow_range=False, per_axis=False))]
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=activ_1, per_axis=False, symmetric=False, narrow_range=False)]

class ModifiedActivation_2QuantizeConfig(DefaultActivationQuantizeConfig):
    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=activ_2, symmetric=False, narrow_range=False, per_axis=False))]
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=activ_2, per_axis=False, symmetric=False, narrow_range=False)]
    
#ModifiedMaxPool2DQuantizerClasses 
class ModifiedMaxPool2DQuantizeConfig(DefaultMaxPool2DQuantizeConfig):
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=maxpool, per_axis=False, symmetric=False, narrow_range=False)]
    
class ModifiedMaxPool2D_1QuantizeConfig(DefaultMaxPool2DQuantizeConfig):
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=maxpool_1, per_axis=False, symmetric=False, narrow_range=False)]
    
class ModifiedMaxPool2D_2QuantizeConfig(DefaultMaxPool2DQuantizeConfig):
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=maxpool_2, per_axis=False, symmetric=False, narrow_range=False)]
    
#ModifiedFlattenQuantizerClasses     
class ModifiedFlattenQuantizeConfig(DefaultFlattenQuantizeConfig):
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=flatten, per_axis=False, symmetric=False, narrow_range=False)]
    
#ModifiedDropoutQuantizerClasses     
class ModifiedDropoutQuantizeConfig(DefaultDropoutQuantizeConfig):
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=dropout, per_axis=False, symmetric=False, narrow_range=False)]   

#ModifiedDenseQuantizerClasses     
class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=dense, symmetric=False, narrow_range=False, per_axis=False))]
    def get_weights_and_quantizers(self, layer):
        return [(layer.kernel, MovingAverageQuantizer(num_bits=dense, symmetric=True, narrow_range=False, per_axis=False))]    
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=dense, per_axis=False, symmetric=False, narrow_range=False)]
    
class ModifiedDense_1QuantizeConfig(DefaultDenseQuantizeConfig):
    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=dense_1, symmetric=False, narrow_range=False, per_axis=False))]
    def get_weights_and_quantizers(self, layer):
        return [(layer.kernel, MovingAverageQuantizer(num_bits=dense_1, symmetric=True, narrow_range=False, per_axis=False))]    
    def get_output_quantizers(self, layer):
        return [quantizers.MovingAverageQuantizer(num_bits=dense_1, per_axis=False, symmetric=False, narrow_range=False)]    
    

#FunctionsForAnnotation    
def apply_quantization_to_layer_Conv2D(layer):
    if isinstance(layer, tf.keras.layers.Conv2D):
        if layer.name == "conv2d":
            return quantize_annotate_layer(layer, ModifiedConv2DQuantizeConfig())
        if layer.name == "conv2d_1":
            return quantize_annotate_layer(layer, ModifiedConv2D_1QuantizeConfig())
        if layer.name == "conv2d_2":
            return quantize_annotate_layer(layer, ModifiedConv2D_2QuantizeConfig())
    return layer

def apply_quantization_to_layer_Activation(layer):
    if isinstance(layer, tf.keras.layers.Activation):
        if layer.name == "activation":
            return quantize_annotate_layer(layer, ModifiedActivationQuantizeConfig())
        if layer.name == "activation_1":
            return quantize_annotate_layer(layer, ModifiedActivation_1QuantizeConfig())
        if layer.name == "activation_2":
            return quantize_annotate_layer(layer, ModifiedActivation_2QuantizeConfig())
    return layer

def apply_quantization_to_layer_MaxPool2D(layer):
    if isinstance(layer, tf.keras.layers.MaxPool2D):
        if layer.name == "max_pooling2d":
            return quantize_annotate_layer(layer,ModifiedMaxPool2DQuantizeConfig())
        if layer.name == "max_pooling2d_1":
            return quantize_annotate_layer(layer, ModifiedMaxPool2D_1QuantizeConfig())
        if layer.name == "max_pooling2d_2":
            return quantize_annotate_layer(layer, ModifiedMaxPool2D_2QuantizeConfig())
    return layer

def apply_quantization_to_layer_Flatten(layer):
    if isinstance(layer, tf.keras.layers.Flatten):
        return quantize_annotate_layer(layer, ModifiedFlattenQuantizeConfig())
    return layer

def apply_quantization_to_layer_Dropout(layer):
    if isinstance(layer, tf.keras.layers.Dropout):
        return quantize_annotate_layer(layer, ModifiedDropoutQuantizeConfig())
    return layer

def apply_quantization_to_layer_Dense(layer):
    if isinstance(layer, tf.keras.layers.Dense):
        if layer.name == "dense":
            return quantize_annotate_layer(layer,ModifiedDenseQuantizeConfig())
        if layer.name == "dense_1":
            return quantize_annotate_layer(layer, ModifiedDense_1QuantizeConfig())
    return layer

#Loading pre-trained model
pre_trained_model = tf.keras.models.load_model("Masks.h5")

print (pre_trained_model.layers[0].name) 

pre_trained_model.summary()

#Quantizing Conv2D
annotated_model1= tf.keras.models.clone_model(
    pre_trained_model,
    clone_function=apply_quantization_to_layer_Conv2D,
) 

with quantize_scope({'ModifiedConv2DQuantizeConfig': ModifiedConv2DQuantizeConfig},
                    {'ModifiedConv2D_1QuantizeConfig': ModifiedConv2D_1QuantizeConfig},
                    {'ModifiedConv2D_2QuantizeConfig': ModifiedConv2D_2QuantizeConfig},
                    {'ModifiedActivationQuantizeConfig': ModifiedActivationQuantizeConfig},
                    {'ModifiedActivation_1QuantizeConfig': ModifiedActivation_1QuantizeConfig},
                    {'ModifiedActivation_2QuantizeConfig': ModifiedActivation_2QuantizeConfig},
                    {'ModifiedMaxPool2DQuantizeConfig': ModifiedMaxPool2DQuantizeConfig},
                    {'ModifiedMaxPool2D_1QuantizeConfig': ModifiedMaxPool2D_1QuantizeConfig},
                    {'ModifiedMaxPool2D_2QuantizeConfig': ModifiedMaxPool2D_2QuantizeConfig},
                    {'ModifiedFlattenQuantizeConfig': ModifiedFlattenQuantizeConfig},
                    {'ModifiedDropoutQuantizeConfig': ModifiedDropoutQuantizeConfig},
                    {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig},
                    {'ModifiedDense_1QuantizeConfig': ModifiedDense_1QuantizeConfig}):
    quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model1)
    
#Quantizing Activation    
annotated_model2= tf.keras.models.clone_model(
    quant_aware_model,
    clone_function=apply_quantization_to_layer_Activation,
)     
    
with quantize_scope({'ModifiedConv2DQuantizeConfig': ModifiedConv2DQuantizeConfig},
                    {'ModifiedConv2D_1QuantizeConfig': ModifiedConv2D_1QuantizeConfig},
                    {'ModifiedConv2D_2QuantizeConfig': ModifiedConv2D_2QuantizeConfig},
                    {'ModifiedActivationQuantizeConfig': ModifiedActivationQuantizeConfig},
                    {'ModifiedActivation_1QuantizeConfig': ModifiedActivation_1QuantizeConfig},
                    {'ModifiedActivation_2QuantizeConfig': ModifiedActivation_2QuantizeConfig},
                    {'ModifiedMaxPool2DQuantizeConfig': ModifiedMaxPool2DQuantizeConfig},
                    {'ModifiedMaxPool2D_1QuantizeConfig': ModifiedMaxPool2D_1QuantizeConfig},
                    {'ModifiedMaxPool2D_2QuantizeConfig': ModifiedMaxPool2D_2QuantizeConfig},
                    {'ModifiedFlattenQuantizeConfig': ModifiedFlattenQuantizeConfig},
                    {'ModifiedDropoutQuantizeConfig': ModifiedDropoutQuantizeConfig},
                    {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig},
                    {'ModifiedDense_1QuantizeConfig': ModifiedDense_1QuantizeConfig}):
    quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model2)
    
#Quantizing MaxPool2D   
annotated_model3= tf.keras.models.clone_model(
    quant_aware_model,
    clone_function=apply_quantization_to_layer_MaxPool2D,
)     
    
with quantize_scope({'ModifiedConv2DQuantizeConfig': ModifiedConv2DQuantizeConfig},
                    {'ModifiedConv2D_1QuantizeConfig': ModifiedConv2D_1QuantizeConfig},
                    {'ModifiedConv2D_2QuantizeConfig': ModifiedConv2D_2QuantizeConfig},
                    {'ModifiedActivationQuantizeConfig': ModifiedActivationQuantizeConfig},
                    {'ModifiedActivation_1QuantizeConfig': ModifiedActivation_1QuantizeConfig},
                    {'ModifiedActivation_2QuantizeConfig': ModifiedActivation_2QuantizeConfig},
                    {'ModifiedMaxPool2DQuantizeConfig': ModifiedMaxPool2DQuantizeConfig},
                    {'ModifiedMaxPool2D_1QuantizeConfig': ModifiedMaxPool2D_1QuantizeConfig},
                    {'ModifiedMaxPool2D_2QuantizeConfig': ModifiedMaxPool2D_2QuantizeConfig},
                    {'ModifiedFlattenQuantizeConfig': ModifiedFlattenQuantizeConfig},
                    {'ModifiedDropoutQuantizeConfig': ModifiedDropoutQuantizeConfig},
                    {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig},
                    {'ModifiedDense_1QuantizeConfig': ModifiedDense_1QuantizeConfig}):
    quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model3) 
    
#Quantizing Flatten   
annotated_model4= tf.keras.models.clone_model(
    quant_aware_model,
    clone_function=apply_quantization_to_layer_Flatten,
)     
    
with quantize_scope({'ModifiedConv2DQuantizeConfig': ModifiedConv2DQuantizeConfig},
                    {'ModifiedConv2D_1QuantizeConfig': ModifiedConv2D_1QuantizeConfig},
                    {'ModifiedConv2D_2QuantizeConfig': ModifiedConv2D_2QuantizeConfig},
                    {'ModifiedActivationQuantizeConfig': ModifiedActivationQuantizeConfig},
                    {'ModifiedActivation_1QuantizeConfig': ModifiedActivation_1QuantizeConfig},
                    {'ModifiedActivation_2QuantizeConfig': ModifiedActivation_2QuantizeConfig},
                    {'ModifiedMaxPool2DQuantizeConfig': ModifiedMaxPool2DQuantizeConfig},
                    {'ModifiedMaxPool2D_1QuantizeConfig': ModifiedMaxPool2D_1QuantizeConfig},
                    {'ModifiedMaxPool2D_2QuantizeConfig': ModifiedMaxPool2D_2QuantizeConfig},
                    {'ModifiedFlattenQuantizeConfig': ModifiedFlattenQuantizeConfig},
                    {'ModifiedDropoutQuantizeConfig': ModifiedDropoutQuantizeConfig},
                    {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig},
                    {'ModifiedDense_1QuantizeConfig': ModifiedDense_1QuantizeConfig}):
    quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model4)
    
#Quantizing Dropout   
annotated_model5= tf.keras.models.clone_model(
    quant_aware_model,
    clone_function=apply_quantization_to_layer_Dropout,
)     
    
with quantize_scope({'ModifiedConv2DQuantizeConfig': ModifiedConv2DQuantizeConfig},
                    {'ModifiedConv2D_1QuantizeConfig': ModifiedConv2D_1QuantizeConfig},
                    {'ModifiedConv2D_2QuantizeConfig': ModifiedConv2D_2QuantizeConfig},
                    {'ModifiedActivationQuantizeConfig': ModifiedActivationQuantizeConfig},
                    {'ModifiedActivation_1QuantizeConfig': ModifiedActivation_1QuantizeConfig},
                    {'ModifiedActivation_2QuantizeConfig': ModifiedActivation_2QuantizeConfig},
                    {'ModifiedMaxPool2DQuantizeConfig': ModifiedMaxPool2DQuantizeConfig},
                    {'ModifiedMaxPool2D_1QuantizeConfig': ModifiedMaxPool2D_1QuantizeConfig},
                    {'ModifiedMaxPool2D_2QuantizeConfig': ModifiedMaxPool2D_2QuantizeConfig},
                    {'ModifiedFlattenQuantizeConfig': ModifiedFlattenQuantizeConfig},
                    {'ModifiedDropoutQuantizeConfig': ModifiedDropoutQuantizeConfig},
                    {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig},
                    {'ModifiedDense_1QuantizeConfig': ModifiedDense_1QuantizeConfig}):
    quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model5)
    
#Quantizing Dense   
annotated_model6= tf.keras.models.clone_model(
    quant_aware_model,
    clone_function=apply_quantization_to_layer_Dense,
)     
    
with quantize_scope({'ModifiedConv2DQuantizeConfig': ModifiedConv2DQuantizeConfig},
                    {'ModifiedConv2D_1QuantizeConfig': ModifiedConv2D_1QuantizeConfig},
                    {'ModifiedConv2D_2QuantizeConfig': ModifiedConv2D_2QuantizeConfig},
                    {'ModifiedActivationQuantizeConfig': ModifiedActivationQuantizeConfig},
                    {'ModifiedActivation_1QuantizeConfig': ModifiedActivation_1QuantizeConfig},
                    {'ModifiedActivation_2QuantizeConfig': ModifiedActivation_2QuantizeConfig},
                    {'ModifiedMaxPool2DQuantizeConfig': ModifiedMaxPool2DQuantizeConfig},
                    {'ModifiedMaxPool2D_1QuantizeConfig': ModifiedMaxPool2D_1QuantizeConfig},
                    {'ModifiedMaxPool2D_2QuantizeConfig': ModifiedMaxPool2D_2QuantizeConfig},
                    {'ModifiedFlattenQuantizeConfig': ModifiedFlattenQuantizeConfig},
                    {'ModifiedDropoutQuantizeConfig': ModifiedDropoutQuantizeConfig},
                    {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig},
                    {'ModifiedDense_1QuantizeConfig': ModifiedDense_1QuantizeConfig}):
    quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model6)    
    
    
quant_aware_model.summary()

#Compiling the quantized model
sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True)
optimizer = sgd
   
quant_aware_model.compile(optimizer=optimizer,
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])


#TrainingTheQuantizedModel

LEARNING_RATE = 0.01
ITERATIONS = 50
BATCH_SIZE = 64
DISPLAY_STEP = 1
CLASSES = 7
CHANNELS = 1
IMAGE_SIZE = 96
N_INPUT = IMAGE_SIZE * IMAGE_SIZE
DROPOUT = 0.5

train_dir = os.path.join(Thepath, 'train')
val_dir = os.path.join(Thepath, 'val')

image_gen_train = ImageDataGenerator(rescale=1./255, 
                                     featurewise_center=False,
                                     featurewise_std_normalization=False,
                                     rotation_range=10,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     zoom_range=.1,
                                     horizontal_flip=True)

train_data_gen = image_gen_train.flow_from_directory(
                                                batch_size=BATCH_SIZE,
                                                directory=train_dir,
                                                shuffle=True,
                                                target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                                class_mode='sparse'
                                                )

image_gen_val = ImageDataGenerator(rescale=1./255) 

val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=val_dir,
                                                 target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                 class_mode='sparse')


checkpoint = ModelCheckpoint('lastchance.h5', monitor='val_accuracy', verbose=1,
    save_best_only=True, mode='auto', period=1)
quant_checkpoint = ModelCheckpoint('New-Quantized-Vizards-3332242.h5', monitor='val_accuracy', verbose=1,
    save_best_only=True, mode='auto', period=1)

history = pre_trained_model.fit(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=10,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE))),
    callbacks=[checkpoint]
)

history = quant_aware_model.fit(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=850,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE))),
    callbacks=[quant_checkpoint]    
)


pre_trained_model_accuracy = pre_trained_model.evaluate_generator(val_data_gen)
quant_aware_model_accuracy = quant_aware_model.evaluate_generator(val_data_gen)

print('Model accuracy:', pre_trained_model_accuracy[1])
print('Quantized Model accuracy:', quant_aware_model_accuracy[1])


conv2d
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 92, 92, 64)        4864      
_________________________________________________________________
activation (Activation)      (None, 92, 92, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 128)       204928    
_________________________________________________________________
activation_1 (Activation)    (None, 42, 42, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256) 

Epoch 3/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1429
Epoch 00003: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 429ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 4/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1367
Epoch 00004: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 438ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 5/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1276
Epoch 00005: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 431ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 6/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1429
Epoch 00006: val_accuracy did not improve from 0.13433
5/5 [================

Epoch 33/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1429
Epoch 00033: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 437ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 34/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1429
Epoch 00034: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 401ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 35/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1276
Epoch 00035: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 408ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 36/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1276
Epoch 00036: val_accuracy did not improve from 0.13433
5/5 [============

Epoch 63/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1429
Epoch 00063: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 397ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 64/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1276
Epoch 00064: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 370ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 65/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1367
Epoch 00065: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 426ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 66/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1367
Epoch 00066: val_accuracy did not improve from 0.13433
5/5 [============

Epoch 93/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1367
Epoch 00093: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 418ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 94/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1367
Epoch 00094: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 408ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 95/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1582
Epoch 00095: val_accuracy did not improve from 0.13433
5/5 [==============================] - 2s 406ms/step - loss: 1.9459 - accuracy: 0.1385 - val_loss: 1.9459 - val_accuracy: 0.1343
Epoch 96/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9459 - accuracy: 0.1276
Epoch 00096: val_accuracy did not improve from 0.13433
5/5 [============

...

Epoch 713/850
4/5 [=======================>......] - ETA: 0s - loss: 1.3408 - accuracy: 0.7500
Epoch 00713: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 414ms/step - loss: 1.3409 - accuracy: 0.7538 - val_loss: 1.1999 - val_accuracy: 0.7761
Epoch 714/850
4/5 [=======================>......] - ETA: 0s - loss: 1.2032 - accuracy: 0.8571
Epoch 00714: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 405ms/step - loss: 1.2434 - accuracy: 0.8269 - val_loss: 1.1680 - val_accuracy: 0.8507
Epoch 715/850
4/5 [=======================>......] - ETA: 0s - loss: 1.2082 - accuracy: 0.8469
Epoch 00715: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 428ms/step - loss: 1.2180 - accuracy: 0.8577 - val_loss: 1.1628 - val_accuracy: 0.8358
Epoch 716/850
4/5 [=======================>......] - ETA: 0s - loss: 1.2017 - accuracy: 0.8520
Epoch 00716: val_accuracy did not improve from 0.91045
5/5 [========

Epoch 743/850
4/5 [=======================>......] - ETA: 0s - loss: 1.8036 - accuracy: 0.3010
Epoch 00743: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 401ms/step - loss: 1.8514 - accuracy: 0.2692 - val_loss: 1.8985 - val_accuracy: 0.2239
Epoch 744/850
4/5 [=======================>......] - ETA: 0s - loss: 1.8471 - accuracy: 0.2653
Epoch 00744: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 398ms/step - loss: 1.8722 - accuracy: 0.2577 - val_loss: 1.7616 - val_accuracy: 0.3284
Epoch 745/850
4/5 [=======================>......] - ETA: 0s - loss: 1.7204 - accuracy: 0.2908
Epoch 00745: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 401ms/step - loss: 1.7747 - accuracy: 0.2577 - val_loss: 1.8326 - val_accuracy: 0.3433
Epoch 746/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9093 - accuracy: 0.1531
Epoch 00746: val_accuracy did not improve from 0.91045
5/5 [========

Epoch 773/850
4/5 [=======================>......] - ETA: 0s - loss: 1.8573 - accuracy: 0.2245
Epoch 00773: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 399ms/step - loss: 1.8801 - accuracy: 0.2346 - val_loss: 1.9025 - val_accuracy: 0.2090
Epoch 774/850
4/5 [=======================>......] - ETA: 0s - loss: 1.8882 - accuracy: 0.2143
Epoch 00774: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 405ms/step - loss: 1.9116 - accuracy: 0.2154 - val_loss: 1.9024 - val_accuracy: 0.2090
Epoch 775/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9010 - accuracy: 0.1990
Epoch 00775: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 407ms/step - loss: 1.9116 - accuracy: 0.2154 - val_loss: 1.9024 - val_accuracy: 0.2090
Epoch 776/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9652 - accuracy: 0.2109
Epoch 00776: val_accuracy did not improve from 0.91045
5/5 [========

Epoch 803/850
4/5 [=======================>......] - ETA: 0s - loss: 1.8865 - accuracy: 0.2143
Epoch 00803: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 413ms/step - loss: 1.9099 - accuracy: 0.2154 - val_loss: 1.9016 - val_accuracy: 0.2090
Epoch 804/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9608 - accuracy: 0.2109
Epoch 00804: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 454ms/step - loss: 1.9082 - accuracy: 0.2154 - val_loss: 1.8996 - val_accuracy: 0.2090
Epoch 805/850
4/5 [=======================>......] - ETA: 0s - loss: 1.8872 - accuracy: 0.2143
Epoch 00805: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 406ms/step - loss: 1.9066 - accuracy: 0.2192 - val_loss: 1.8975 - val_accuracy: 0.2090
Epoch 806/850
4/5 [=======================>......] - ETA: 0s - loss: 1.9577 - accuracy: 0.2109
Epoch 00806: val_accuracy did not improve from 0.91045
5/5 [========

Epoch 833/850
3/5 [=================>............] - ETA: 0s - loss: 1.9651 - accuracy: 0.2083
Epoch 00833: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 395ms/step - loss: 1.9099 - accuracy: 0.2154 - val_loss: 1.9009 - val_accuracy: 0.2090
Epoch 834/850
4/5 [=======================>......] - ETA: 0s - loss: 1.8786 - accuracy: 0.2245
Epoch 00834: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 413ms/step - loss: 1.9099 - accuracy: 0.2154 - val_loss: 1.9009 - val_accuracy: 0.2090
Epoch 835/850
4/5 [=======================>......] - ETA: 0s - loss: 1.8785 - accuracy: 0.2245
Epoch 00835: val_accuracy did not improve from 0.91045
5/5 [==============================] - 2s 401ms/step - loss: 1.9098 - accuracy: 0.2154 - val_loss: 1.9009 - val_accuracy: 0.2090
Epoch 836/850
4/5 [=======================>......] - ETA: 0s - loss: 1.8869 - accuracy: 0.2143
Epoch 00836: val_accuracy did not improve from 0.91045
5/5 [========